# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [260]:
! pip install schedule pytest BeautifulSoup4 pandas requests_futures 

In [261]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
try:
    import time
    import requests
    import schedule
    from bs4 import BeautifulSoup
    import pytest
    import re
    import pandas as pd
    import json
    from requests_futures.sessions import FuturesSession
    from concurrent.futures import ThreadPoolExecutor
    from threading import local
    print("Все зависимости готовы!")
except ImportError as e:
    print(f"Ошибка импорта: {e}")

Все зависимости готовы!


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [263]:
def get_book_data(book_url: str) -> dict:
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ


    session = requests.Session()

    try:
        
        page = session.get(book_url)
        page.raise_for_status()
        page.encoding = "utf-8"
        soup = BeautifulSoup(page.text, "html.parser")

        product_dict = {}

        name = soup.find("div", class_="col-sm-6 product_main").find("h1").get_text()
        product_dict["Name"] = name

        stars = (
            soup.find("p", class_="star-rating")
            .get("class")[1]
            .replace("One", "1")
            .replace("Two", "2")
            .replace("Three", "3")
            .replace("Four", "4")
            .replace("Five", "5")
        )
        product_dict["Rating"] = stars

        description = (
            soup.find("div", id="product_description", class_="sub-header")
            .find_next_sibling()
            .get_text()
        )
        product_dict["Description"] = description

        table = soup.find("table", class_="table table-striped").find_all("tr")
        for row in table:
            product_dict[row.find("th").get_text().strip()] = (
                row.find("td").get_text().strip()
            )

        return product_dict

    except requests.RequestException as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return {}
    
    
    finally:
        session.close()

# КОНЕЦ ВАШЕГО РЕШЕНИЯ


In [264]:
# Используйте для самопроверки
book_url = "http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html"
print(get_book_data(book_url))


{'Name': 'A Light in the Attic', 'Rating': '3', 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I thi

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [262]:
def scrape_books(catalog_url: str, is_save: bool =False) -> dict:
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ



    # session = FuturesSession(max_workers=10)
    # my_session = session()
    # future_session = FuturesSession(session=my_session)
    session = requests.Session()


    try:
        big_list =[]
        page_url = catalog_url
        upper_page_url = '/'.join(catalog_url.split('/')[:-1])
        
        # curent_page_numb = soup.find("li", class_="current").get_text().split()[1]
        # max_pages_numb = soup.find("li", class_="current").get_text().split()[-1]
        # page_url = re.sub(r'\d+', str(int(curent_page_numb) + 1), page_url)    #curent_page.replace(curent_page_numb, int(curent_page_numb) + 1)
        # print(page_url, curent_page_numb, max_pages_numb)




        while True:



            curent_page = session.get(page_url)
            curent_page.raise_for_status()
            curent_page.encoding = 'utf-8'
            soup = BeautifulSoup(curent_page.text, "html.parser")

#            product_table_dict = {}

            curent_page_numb = soup.find("li", class_="current").get_text().split()[1]
            max_pages_numb = soup.find("li", class_="current").get_text().split()[-1]


            curent_page_link_list = []
            links = soup.find_all("a", title=True)
            for link in links:
                href = link.get("href")
                if href:
                    
                    absolute_url = upper_page_url + "/" + href
                    curent_page_link_list.append(absolute_url)


            print(page_url, curent_page_numb, max_pages_numb)

            print(curent_page_link_list)
            for book_url in curent_page_link_list:
                big_list.append(get_book_data(book_url))


            if curent_page_numb == "3": #max_pages_numb:
                break
            
            page_url = re.sub(r'\d+', str(int(curent_page_numb) + 1), page_url)    #curent_page.replace(curent_page_numb, int(curent_page_numb) + 1)
            

#        print(big_list)
        return(big_list)

    except requests.RequestException as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return {}
    
    finally:
        session.close()



    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [265]:
catalog_url = 'http://books.toscrape.com/catalogue/page-1.html'
#scrape_books(catalog_url)
#print(scrape_books(catalog_url))


#data_table = pd.DataFrame(scrape_books(catalog_url))


result = scrape_books(catalog_url)
data_table = pd.json_normalize(result)



# with open("data.txt", "w") as file:
#         file.write(data_table)
print(data_table)
#d[d[0] == 'Страна'][1].values[0]

http://books.toscrape.com/catalogue/page-1.html 1 50
['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html', 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html', 'http://books.toscrape.com/catalogue/soumission_998/index.html', 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html', 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html', 'http://books.toscrape.com/catalogue/the-requiem-red_995/index.html', 'http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html', 'http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html', 'http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html', 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html', 'http://books.toscrape.com/catalogue/sta

In [ ]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```